<a href="https://colab.research.google.com/github/Sikandarh11/CodeClause-Data-Science-Internship-/blob/main/Movie%20Recommendation%20System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d rounakbanik/the-movies-dataset

the-movies-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/the-movies-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ratings = pd.read_csv('/content/ratings_small.csv')
movies = pd.read_csv("/content/movies_metadata.csv")

<ipython-input-45-6f3b89c25b2d>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("/content/movies_metadata.csv")


In [ ]:
ratings.shape, movies.shape

((100004, 4), (45466, 24))

In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [122]:
moviesdf = movies[['id', 'genres', 'original_title', 'vote_average', 'vote_count', 'homepage']]
moviesdf.isnull().sum()

id                    0
genres                0
original_title        0
vote_average          6
vote_count            6
homepage          37684
dtype: int64

In [123]:
moviesdf = moviesdf.dropna(axis=0)

In [124]:
moviesdf.isnull().sum()

id                0
genres            0
original_title    0
vote_average      0
vote_count        0
homepage          0
dtype: int64

In [125]:
moviesdf.head()

,id,genres,original_title,vote_average,vote_count,homepage
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,7.7,5415.0,http://toystory.disney.com/toy-story
9,710,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",GoldenEye,6.6,1194.0,http://www.mgm.com/view/movie/757/Goldeneye/
24,451,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",Leaving Las Vegas,7.1,365.0,http://www.mgm.com/title_title.do?title_star=L...
46,807,"[{'id': 80, 'name': 'Crime'}, {'id': 9648, 'na...",Se7en,8.1,5915.0,http://www.sevenmovie.com/
49,629,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",The Usual Suspects,8.1,3334.0,http://www.mgm.com/#/our-titles/2083/The-Usual...


In [126]:
moviesdf.rename(columns = {"id": "movieId"}, inplace = True)
moviesdf.head()

,movieId,genres,original_title,vote_average,vote_count,homepage
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,7.7,5415.0,http://toystory.disney.com/toy-story
9,710,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",GoldenEye,6.6,1194.0,http://www.mgm.com/view/movie/757/Goldeneye/
24,451,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",Leaving Las Vegas,7.1,365.0,http://www.mgm.com/title_title.do?title_star=L...
46,807,"[{'id': 80, 'name': 'Crime'}, {'id': 9648, 'na...",Se7en,8.1,5915.0,http://www.sevenmovie.com/
49,629,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",The Usual Suspects,8.1,3334.0,http://www.mgm.com/#/our-titles/2083/The-Usual...


In [127]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [128]:
ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [129]:
moviesdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7778 entries, 0 to 45461
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   movieId         7778 non-null   object 
 1   genres          7778 non-null   object 
 2   original_title  7778 non-null   object 
 3   vote_average    7778 non-null   float64
 4   vote_count      7778 non-null   float64
 5   homepage        7778 non-null   object 
dtypes: float64(2), object(4)
memory usage: 425.4+ KB


In [130]:
moviesdf['movieId'] = moviesdf['movieId'].astype(int)

In [131]:
df = ratings.merge(moviesdf, on = 'movieId')
df.head()

,userId,movieId,rating,timestamp,genres,original_title,vote_average,vote_count,homepage
0,1,2193,2.0,1260759198,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",My Tutor,5.8,17.0,http://www.crownintlpictures.com/lntitles.html
1,4,2193,3.0,949896070,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",My Tutor,5.8,17.0,http://www.crownintlpictures.com/lntitles.html
2,30,2193,3.0,945294700,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",My Tutor,5.8,17.0,http://www.crownintlpictures.com/lntitles.html
3,33,2193,2.0,1032680004,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",My Tutor,5.8,17.0,http://www.crownintlpictures.com/lntitles.html
4,57,2193,5.0,907764431,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",My Tutor,5.8,17.0,http://www.crownintlpictures.com/lntitles.html


In [132]:
df.shape

(10959, 9)

In [137]:
y = df.groupby('original_title').count()['vote_count'] > (100)
vote_count_index = y[y].index
filtered_data = df[df['original_title'].isin(vote_count_index)]
filtered_data.shape

(3132, 9)

In [139]:
x = filtered_data.groupby('userId').count()['vote_count'] > (15)
vote_count_index = x[x].index
filtered_data_f = filtered_data[filtered_data['userId'].isin(vote_count_index)]
filtered_data_f.shape

(545, 9)

In [140]:
pt = filtered_data_f.pivot_table(index = 'original_title', columns = 'userId', values = 'rating')
pt.fillna( 0, inplace = True)
pt

userId,15,19,30,73,77,88,105,130,134,212,...,509,518,547,564,580,596,607,624,654,665
original_title,,,,,,,,,,,,,,,,,,,,,
Back to the Future Part II,3.0,3.0,4.0,3.5,2.5,3.0,3.5,2.0,4.0,3.0,...,0.0,3.0,4.5,4.0,0.0,3.5,3.5,4.0,4.0,3.0
Bang Boom Bang - Ein todsicheres Ding,2.0,3.0,2.0,2.0,3.0,2.0,3.0,0.0,2.5,1.0,...,2.0,4.0,0.5,3.0,3.0,3.5,2.5,3.0,4.5,3.0
Blood: The Last Vampire,3.0,0.0,4.0,4.0,4.5,4.0,0.0,3.5,0.0,3.5,...,4.0,4.0,5.0,3.0,4.0,0.0,0.0,4.0,5.0,4.0
Bridge to Terabithia,4.0,4.0,4.0,4.0,4.0,2.5,4.0,4.5,3.5,2.0,...,2.0,3.0,5.0,0.0,3.5,4.0,4.5,4.0,4.0,2.0
Crustacés et coquillages,3.0,3.0,5.0,3.0,3.5,0.0,3.0,3.0,4.0,0.0,...,3.0,0.0,3.5,4.0,0.0,0.0,4.0,4.0,4.0,0.0
Dave Chappelle's Block Party,1.0,4.0,0.0,3.0,2.5,0.0,3.5,0.0,4.5,3.5,...,0.0,4.0,0.0,1.0,3.0,3.5,3.5,0.0,4.5,4.0
Dawn of the Dead,5.0,4.0,4.0,1.5,3.5,4.0,0.0,0.0,4.0,0.0,...,4.0,3.0,5.0,3.0,3.0,3.5,0.0,4.0,4.0,0.0
Jurassic Park,1.5,3.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,0.0,...,0.0,3.0,0.0,3.0,2.0,3.5,3.5,4.0,4.0,0.0
Knallhart,3.0,3.0,4.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,...,2.0,4.0,1.5,4.0,2.0,3.0,3.5,3.0,4.0,0.0


In [141]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_score = cosine_similarity(pt)

In [142]:
def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_score[index])),key=lambda x:x[1],reverse=True)[1:5]

    data = []
    for i in similar_items:
        item = []
        temp_df = moviesdf[moviesdf['original_title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('original_title')['original_title'].values))
        item.extend(list(temp_df.drop_duplicates('original_title')['homepage'].values))
       # item.extend(list(temp_df.drop_duplicates('original_title')['Image-URL-M'].values))

        data.append(item)

    return data

In [146]:
recommend('The Talented Mr. Ripley')

[['Bridge to Terabithia',
  'http://disney.go.com/disneyvideos/liveaction/bridgetoterabithia/'],
 ['Men in Black II', 'http://www.sonypictures.com/homevideo/meninblackii/'],
 ['Romeo + Juliet', 'http://www.romeoandjuliet.com/'],
 ['Crustacés et coquillages', 'http://www.crustaces-lefilm.com/']]

In [147]:
import pickle
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(moviesdf,open('books.pkl','wb'))
pickle.dump(similarity_score,open('similarity_scores.pkl','wb'))